In [ ]:
import polyfingerprints as pfp 
import pandas as pd
import numpy as np
from wrapconfig import YAMLWrapConfig
import matplotlib.pyplot as plt

In [ ]:
RAW_CSV_PATH =  "./cloud_points_data.csv"

In [ ]:
raw_df = pd.read_csv(RAW_CSV_PATH, sep=";", decimal=",")
raw_df

In [ ]:
raw_df["pH"] = raw_df["pH"].fillna(7)
DEF_TYPE_MAP = {
        "A": 0.1,
        "B": 0.2,
        "C": 0.5,
        "DSC": 0.01,
    }
raw_df["def_type"] = raw_df["def_type"].fillna("0.1")
raw_df["def_type"] = raw_df["def_type"].apply(
    lambda x: float(
        (DEF_TYPE_MAP[x] if x in DEF_TYPE_MAP else str(x).replace(",", "."))
    ),
)

# set poly_conc to polymer_concentration_mass_conc where it is NaN
if "polymer_concentration_mass_conc" in raw_df.columns:
    raw_df["polymer_concentration_wpercent"] = raw_df["polymer_concentration_wpercent"].fillna(raw_df["polymer_concentration_mass_conc"])
    raw_df= raw_df.drop(
        ["polymer_concentration_mass_conc"], axis=1
    )

df,info = pfp.datareader.expand_data(
    raw_df,
    ignored_columns=["reference",
        "polymer_type",
        "polymer_type_style",
        "polymer_architecture",
        "polymerisation_type",
        "Mw",
        "PDI",
        "mass_characterisation_method",
        "mass_characterisation_standart",
        "N/A",
        "identifier",
        "comment",
        "tacticity",
        "rating",],
    density=1,
additive_wp_column_postfix = "_concentration_weight_percent",
    additive_conc_column_postfix="_concentration_molar"
    

)

df.to_csv("expanded_data.csv")
infofile = YAMLWrapConfig("expanded_info.yml")
infofile.set_data(info)
infofile.save()
df